You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings

warnings.filterwarnings(action="ignore")

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [4]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = "gpt-4o"

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [5]:
help(request=Agent)

Help on class Agent in module crewai.agent:

class Agent(crewai.agents.agent_builder.base_agent.BaseAgent)
 |  Agent(*, id: Annotated[uuid.UUID, UuidVersion(uuid_version=4)] = <factory>, formatting_errors: int = 0, role: str, goal: str, backstory: str, config: Optional[Dict[str, Any]] = None, cache: bool = True, verbose: bool = False, max_rpm: Optional[int] = None, allow_delegation: bool = False, tools: Optional[List[Any]] = <factory>, max_iter: int = 25, agent_executor: pydantic.functional_validators.InstanceOf = None, llm: Union[str, Annotated[crewai.llm.LLM, InstanceOf()], Any] = None, crew: Any = None, i18n: crewai.utilities.i18n.I18N = I18N(prompt_file=None), cache_handler: Optional[Annotated[crewai.agents.cache.cache_handler.CacheHandler, InstanceOf()]] = None, tools_handler: Annotated[crewai.agents.tools_handler.ToolsHandler, InstanceOf()] = <factory>, max_tokens: Optional[int] = None, knowledge: Optional[crewai.knowledge.knowledge.Knowledge] = None, knowledge_sources: Optional[

In [ ]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
    "about the topic: {topic}."
    "You collect information that helps the "
    "audience learn something "
    "and make informed decisions. "
    "Your work is the basis for "
    "the Content Writer to write an article on this topic.",
    allow_delegation=False,
    verbose=True,
)

### Agent: Writer

In [7]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
    "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
    "a new opinion piece about the topic: {topic}. "
    "You base your writing on the work of "
    "the Content Planner, who provides an outline "
    "and relevant context about the topic. "
    "You follow the main objectives and "
    "direction of the outline, "
    "as provide by the Content Planner. "
    "You also provide objective and impartial insights "
    "and back them up with information "
    "provide by the Content Planner. "
    "You acknowledge in your opinion piece "
    "when your statements are opinions "
    "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True,
)

### Agent: Editor

In [8]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
    "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
    "from the Content Writer. "
    "Your goal is to review the blog post "
    "to ensure that it follows journalistic best practices,"
    "provides balanced viewpoints "
    "when providing opinions or assertions, "
    "and also avoids major controversial topics "
    "or opinions when possible.",
    allow_delegation=False,
    verbose=True,
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [9]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
        "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
        "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
        "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
    "with an outline, audience analysis, "
    "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [10]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
        "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
        "3. Sections/Subtitles are properly named "
        "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
        "engaging introduction, insightful body, "
        "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
        "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
    "in markdown format, ready for publication, "
    "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [11]:
edit = Task(
    description=(
        "Proofread the given blog post for "
        "grammatical errors and "
        "alignment with the brand's voice."
    ),
    expected_output="A well-written blog post in markdown format, "
    "ready for publication, "
    "each section should have 2 or 3 paragraphs.",
    agent=editor,
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [ ]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=True,
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [13]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

# Agent: Content Planner
## Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


# Agent: Content Planner
## Final Answer: 
**Comprehensive Content Plan Document on Artificial Intelligence**

**1. Latest Trends, Key Players, and Noteworthy News on Artificial Intelligence**

- **Latest Trends:**
  - Generative AI: With models like OpenAI's GPT-3 and ChatGPT, generative AI is becoming integral in content creation, coding, and art.
  - AI Ethics and Bias: There's increasing focus on ensuring AI systems are fair and unbiased, with responsible AI development at the forefront.
  - AI in Healthcare: AI is being used for diagnostics, treatment recommendations, and predictive analytics in healthcare.
  - AI and Automati

- Display the results of your execution as markdown in the notebook.

In [ ]:
from IPython.display import Markdown

Markdown(data=result.raw)

```markdown
# Navigating the Future with Artificial Intelligence: Trends, Innovations, and Ethical Imperatives

Artificial Intelligence (AI) stands as a transformative force in the modern world, reshaping industries and everyday experiences alike. From AI-driven chatbots orchestrating customer interactions to sophisticated algorithms diagnosing complex health conditions, AI's influence knows no bounds. This technological marvel has rapidly woven itself into the fabric of contemporary society, heralding a new era of innovation and efficiency across diverse sectors.

In recent years, AI's scope and capabilities have expanded dramatically. Key players like OpenAI and Google DeepMind drive groundbreaking research and innovations, setting the stage for an AI-powered future. This piece delves into the latest trends in AI, explores the contributions of leading companies, examines the challenges and ethical debates surrounding AI, and ponders its future impact on the workforce and sustainability efforts.

## AI Trends Transforming Industries

One of the most captivating trends today is Generative AI, which is making significant strides in fields such as content creation, coding, and digital art. Tools like OpenAI's GPT-3 and ChatGPT facilitate seamless interaction between machines and humans, opening new horizons for creativity and productivity. For instance, these AI models are employed to draft articles, produce music, and even generate code, offering a glimpse into a world where AI assists in human endeavors with unprecedented precision.

In parallel, the ethical implications of AI have commanded attention. Ensuring fairness and mitigating bias in AI systems are critical. Organizations are investing in responsible AI development to address concerns such as data privacy and algorithmic discrimination. Striking a balance between innovation and ethical considerations is imperative to foster trust and acceptance of AI technologies.

Moreover, AI's integration into healthcare promises transformative benefits. From aiding diagnostics through predictive analytics to offering personalized treatment recommendations, AI is revolutionizing patient care. Its potential to streamline healthcare processes heralds a future where AI enhances both accuracy and efficiency, ultimately improving clinical outcomes.

## Leading Companies and Innovations

Top-tier organizations like OpenAI, Google DeepMind, IBM Watson, Amazon Web Services, and Microsoft AI are at the forefront of AI advancement. Their contributions significantly shape the technology's trajectory, influencing AI's role across various domains. OpenAI's innovations in natural language processing, for instance, are redefining human-machine interactions, while Google DeepMind's focus on machine learning and neural networks pushes the envelope of what's possible with AI.

These companies not only pioneer research but also engage in strategic acquisitions and investments in AI startups, signaling a robust commitment to building a future dominated by AI technologies. Collaborative initiatives, especially in healthcare, spearhead breakthroughs that could redefine how diseases are treated and managed, epitomizing the symbiotic relationship between tech giants and the health sector.

## Challenges and Ethical Considerations

Despite AI's promise, it is not without its challenges. AI bias remains an omnipresent concern, potentially leading to unequal outcomes across different user demographics. As AI systems increasingly infiltrate decision-making processes, maintaining data privacy and ensuring ethical use of AI become paramount. Frameworks like the EU's AI Act aim to establish regulations that govern AI development responsibly, ensuring these systems uphold human values and rights.

The ethical debate extends to the ownership and usage of data. As AI systems consume vast amounts of information to learn and improve, the infringement of personal privacy becomes a critical point of discussion. Organizations must transparently navigate these issues, creating AI systems that are both ethical and effective.

## Future of AI and Human Workforce

AI's rapid advancement inevitably raises questions about its impact on employment and required skills. As automation becomes more prevalent, industries may witness a fundamental shift in the job market. While AI can enhance productivity and streamline operations, it may also displace certain roles, prompting a reevaluation of workforce dynamics.

However, this evolution also presents opportunities. New roles focused on AI oversight, ethical management, and technological integration will arise, requiring an upskilled workforce ready to adapt to and leverage AI-driven environments. As AI transforms industries, continuous learning and adaptability will remain essential for individuals and companies alike.

## AI in Sustainability

Harnessing AI for sustainability is a promising frontier. From optimizing energy consumption to predicting environmental changes, AI is pivotal in tackling climate change. Through intelligent systems that enhance resource management and reduce waste, AI contributes to a more sustainable future. Companies are increasingly investing in AI solutions geared towards sustainability, showcasing AI's potential as a catalyst for environmental stewardship and innovation.

## Concluding Thoughts

The rapid evolution of Artificial Intelligence presents a landscape teeming with opportunities and challenges. As AI continues to redefine sectors and redefine possibilities, staying informed and engaged with these advancements is crucial. Embracing AI not only enhances productivity and innovation but also demands a conscientious approach to ethics and sustainability. 

As we chart the course of AI's future, let us commit to fostering an ecosystem where technology serves humanity, aligns with ethical values, and promotes a sustainable world. We invite you to subscribe to our channel for ongoing updates and comprehensive analyses on AI and its burgeoning influence in the digital realm.
```

This well-structured blog post integrates SEO keywords like "Artificial Intelligence Trends 2023," "AI in Healthcare," and "Ethical AI Development" naturally throughout the text, meeting the described criteria for an engaging piece.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [ ]:
topic = "YOUR TOPIC HERE"
result = crew.kickoff(inputs={"topic": topic})

In [ ]:
Markdown(result)

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).